In [8]:
import numpy as np
import pandas as pd
from gensim import models
from gensim.models import Phrases
from gensim.models.fasttext import FastText
from gensim.models.wrappers.fasttext import FastText as FT_wrapper
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential, Model, load_model, save_model
from keras.layers import Dense
from keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Flatten
from keras.layers import GaussianNoise, BatchNormalization, Dropout
from keras.layers import Activation, Input, concatenate, Reshape, merge, dot
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l1, l2
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import skipgrams, make_sampling_table
from keras.callbacks import Callback, LambdaCallback, TensorBoard, ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from random import shuffle
import time
import pylab as pl
from ipywidgets import FloatProgress
from IPython import display
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
def is_desired_letter(char):
    order = ord(char)
    return order >= 97 and order < 123 or order >= 48 and order < 58 or order == ord(" ") or order == ord("'")


def get_text_data():
    # load the dataset but only keep the top n words, zero the rest
    train_data = pd.read_csv("../../input/kickstarter_train.csv")
    # segment all sentences
    sent_list = [sent.lower() for text in train_data["desc"] if type(text) is str for sent in sent_tokenize(text)]
    # remove symbols in each description
    sent_list = [[char for char in sent if is_desired_letter(char)] for sent in sent_list]
    sent_list = [''.join(sent).split() for sent in sent_list]
    # remove too short desc
    train_texts = [sent for sent in sent_list if len(sent) > 3]
    train_data = pd.read_csv("../../input/kickstarter_test.csv")
    # segment all sentences
    sent_list = [sent.lower() for text in train_data["desc"] if type(text) is str for sent in sent_tokenize(text)]
    # remove symbols in each description
    sent_list = [[char for char in sent if is_desired_letter(char)] for sent in sent_list]
    sent_list = [''.join(sent).split() for sent in sent_list]
    # remove too short desc
    train_texts.extend([sent for sent in sent_list if len(sent) > 3])

    return train_texts

In [5]:
# get training texts from disk
try:
    open("../../input/all_sentences.txt", "r")
except:
    print("no existing sentence data, generate data from train data")
    # load sentences from training data
    train_texts = get_text_data()
    print("data of length {} grabbed".format(len(train_texts)))
    # write sentences to file
    with open("../../input/all_sentences.txt", "w") as text_file:
        for sent in train_texts:
            joint_sent = ' '.join(sent)
            text_file.write(joint_sent + '\n ')
    print("sentences are written to all_sentences.txt")

# A lineSentence must be used to generate training data
# Reason remains to be known

In [4]:
ft_home =  "/Users/andywu/anaconda3/envs/tensorflow/lib/python3.5/site-packages/fastText/fasttext"
skipgram_fasttext_model = FT_wrapper.train(corpus_file="../../input/all_sentences.txt", model="skipgram", ft_path=ft_home, sorted_vocab=True, size=64, min_count=15, iter=500, alpha=0.05)
skipgram_fasttext_model.save('w2v_data/fasttext_w2v_64')
skipgram_fasttext_model.wv.save('w2v_data/fasttext_w2v_vector_64')
print(skipgram_fasttext_model)

KeyboardInterrupt: 

In [10]:
model = KeyedVectors.load('w2v_data/fasttext_w2v_128')
model.wv.most_similar(positive=['man', 'king'], negative=['woman'])

[('jr', 0.49340614676475525),
 ('dragon', 0.4417273700237274),
 ('bob', 0.4281129539012909),
 ('jeff', 0.4203910827636719),
 ('burning', 0.4189816415309906),
 ('james', 0.41860032081604004),
 ('nintendo', 0.4155583679676056),
 ('luther', 0.4138791561126709),
 ('legendary', 0.41267311573028564),
 ('murray', 0.4058099389076233)]

In [11]:
model.wv.most_similar_to_given(word_list=['you', 'me'], w1='goo')

'me'

In [12]:
model.wv['me'].dot(model.wv['me'])

8.0347137

In [15]:
'fant' in model.wv.vocab

False

In [20]:
'tic' in model.wv

True